In [ ]:
import numpy as np
import h5py

In [ ]:
from src.load_script import load_contest_train_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from keras.utils import to_categorical
import joblib

save_to_cache = False
load_from_cache = True

if not load_from_cache:
    X, y, samples = load_contest_train_dataset('datasets/contest_TRAIN.h5', 100)
    wavelengths = X.columns

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=samples)
    del X, y, samples
    
    pipe = RobustScaler()

    X_train = pipe.fit_transform(X_train)
    X_test = pipe.transform(X_test)

    y_train, y_test = to_categorical(y_train), to_categorical(y_test)

    if save_to_cache:
        joblib.dump(pipe, 'models/preprocessing_pipeline.pkl')
        np.save(open('datasets/x_train.npy', 'wb'), X_train)
        np.save(open('datasets/y_train.npy', 'wb'), y_train)
        np.save(open('datasets/x_test.npy', 'wb'), X_test)
        np.save(open('datasets/y_test.npy', 'wb'), y_test)
        np.save(open('datasets/wavelengths.npy', 'wb'), wavelengths)
else:
    pipe = joblib.load('models/preprocessing_pipeline.pkl')
    X_train = np.load(open('datasets/x_train.npy', 'rb'))
    y_train = np.load(open('datasets/y_train.npy', 'rb'))
    X_test = np.load(open('datasets/x_test.npy', 'rb'))
    y_test = np.load(open('datasets/y_test.npy', 'rb'))
    wavelengths = np.load(open('datasets/wavelengths.npy', 'rb'))

In [ ]:
from src.visualization import plot_spectra

plot_spectra(pipe.inverse_transform([X_train.mean(axis=0)]), calibration=wavelengths)

In [ ]:
import tensorflow as tf

mlp = tf.keras.Sequential([
  tf.keras.layers.Dense(1024, activation='leaky_relu'),
  tf.keras.layers.Dense(516, activation='leaky_relu'),
  tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid'),
])

mlp.compile(loss='mse', optimizer="adam", metrics=['accuracy'])

mlp.fit(X_train, y_train, batch_size=32, epochs=3, verbose="auto")

In [ ]:
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:
cnn = tf.keras.Sequential([
  tf.keras.layers.Conv1D(8, 20, activation='relu'),
  tf.keras.layers.MaxPooling1D(10),
  tf.keras.layers.Conv1D(8, 20, activation='relu'),
  tf.keras.layers.MaxPooling1D(10),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid'),
])

cnn.compile(loss='mse', optimizer="adam", metrics=['accuracy'])

cnn.fit(X_train, y_train, batch_size=32, epochs=3, verbose="auto")

In [ ]:
rnn = tf.keras.Sequential([
  tf.keras.layers.GRU(4, activation='leaky_relu'),
  tf.keras.layers.Dense(128, activation='leaky_relu'),
  tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid'),
])

rnn.compile(loss='mse', optimizer="adam", metrics=['accuracy'])

rnn.fit(X_train, y_train, batch_size=32, epochs=3)

In [ ]:
combination = tf.keras.Sequential([
  tf.keras.layers.Conv1D(3, 20, activation='relu'),
  tf.keras.layers.MaxPooling1D(2, strides=5),
  tf.keras.layers.GRU(32),
  tf.keras.layers.Dense(64, activation='leaky_relu'),
  tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid'),
])

combination.compile(loss='mse', optimizer="adam", metrics=['accuracy'])

combination.fit(X_train, y_train, batch_size=32, epochs=3)

In [ ]:
"""from keras.layers import Input, Dense, GRU
from keras.models import Model
latent_dim = 4

encoder_inputs = Input(shape=wavelengths.shape)
encoder = GRU(latent_dim, return_state=True)
encoder_outputs, state_h = encoder(encoder_inputs)

decoder_inputs = Input(shape=(1, latent_dim))
decoder_gru = GRU(latent_dim, return_sequences=True)
decoder_outputs = decoder_gru(decoder_inputs, initial_state=state_h)
decoder_dense = Dense(1, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile('adam', 'mse')

model.fit(X_train, y_train)"""